In [2]:
from scipy.stats import chisquare
from scipy.stats import norm
from scipy.stats import lognorm
from scipy.stats import chi2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def get_normal_paramteres(array):
    array = np.array(array)
    mu = np.mean(array)
    sigma = ((1/(np.size(array)-1))*np.sum((array-mu)**2))**0.5
    return (mu,sigma)
def get_log_normal_paramaters(array):
    array = np.array(array)
    mu,sigma = get_normal_paramteres(array)
    shape_paramter = (np.log(1+(sigma/mu)**2))**0.5
    lambda_parameter = np.log(mu)-0.5*(shape_paramter**2)
    return (shape_paramter,lambda_parameter)
def normal_theoritical_frequency(array,bins):
    bins = np.array(bins)
    array = np.array(array)
    mu,sigma = get_normal_paramteres(array)
    dist = norm(mu,sigma)
    theroriticals = []
    for index,value in enumerate(bins):
        if(index == 0):
            theroriticals.append(dist.cdf(value)*np.size(array))
        elif(index == np.size(bins)-1):
            value2 = bins[index-1]
            theroriticals.append(dist.cdf(value)*np.size(array)-dist.cdf(value2)*np.size(array))
            theroriticals.append((1 - dist.cdf(value))*np.size(array))
        else:
            value2 = bins[index-1]
            theroriticals.append(dist.cdf(value)*np.size(array)-dist.cdf(value2)*np.size(array))
    return theroriticals
def lognorm_cdf(lambda_parameter,shape_paramter,x,size):
    a = (np.log(x)-lambda_parameter)/shape_paramter
    return norm(0,1).cdf(a)*size
def log_normal_theoritical_frequency(array,bins):
    bins = np.array(bins)
    array = np.array(array)
    shape_paramter,lambda_parameter = get_log_normal_paramaters(array)
    theroriticals = []
    array_size = np.size(array)
    for index,value in enumerate(bins):
        if(index == 0):
            theroriticals.append(lognorm_cdf(lambda_parameter,shape_paramter,value,array_size))
        elif(index == np.size(bins)-1):
            value2 = bins[index-1]
            theroriticals.append(lognorm_cdf(lambda_parameter,shape_paramter,value,array_size)-lognorm_cdf(lambda_parameter,shape_paramter,value2,array_size))
            theroriticals.append((1*array_size - lognorm_cdf(lambda_parameter,shape_paramter,value,array_size)))
        else:
            value2 = bins[index-1]
            theroriticals.append(lognorm_cdf(lambda_parameter,shape_paramter,value,array_size)-lognorm_cdf(lambda_parameter,shape_paramter,value2,array_size))
    return theroriticals
def get_coditioned_sized(array,condition):
    dataframe = pd.DataFrame(data = {"data_values":array})
    return len(list((dataframe[condition].to_dict()['data_values']).values()))
def observed_array(array,bins):
    bins = np.array(bins)
    array = np.array(array)
    observeds = []
    array_size = np.size(array)
    dataframe = pd.DataFrame(data={"data_values":array})
    for index,value in enumerate(bins):
        if(index == 0):
            condition = dataframe["data_values"]<=value
            observeds.append((get_coditioned_sized(array,condition)))
        elif(index == np.size(bins)-1):
            value2 = bins[index-1]
            condition = (dataframe["data_values"]<=value) & (dataframe["data_values"]>value2)
            observeds.append(get_coditioned_sized(array,condition))
            condition = (dataframe["data_values"]>=value)
            observeds.append(get_coditioned_sized(array,condition))
        else:
            value2 = bins[index-1]
            condition = (dataframe["data_values"]<=value) & (dataframe["data_values"]>value2)
            observeds.append(get_coditioned_sized(array,condition))
    return observeds
def bins_to_string(bins):
    bins = np.array(bins)
    string_bins = [] 
    for index,bin in enumerate(bins):
        if(index == 0):
            string_bins.append("< " + str(bin))
        elif(index == np.size(bins)-1):
            bin2 = bins[index-1]
            string_bins.append(str(bin2) + " - " + str(bin))
            string_bins.append(str(bin) + " >")
        else:
            bin2 = bins[index-1]
            string_bins.append(str(bin2) + " - " + str(bin))
    return string_bins
def make_dataframe(array,bins):
    observeds = observed_array(array,bins)
    theo_log_normal = log_normal_theoritical_frequency(array,bins)
    theo_normal = normal_theoritical_frequency(array,bins)
    string_bins = bins_to_string(bins)
    dataframe = pd.DataFrame(data = {"Interval" : string_bins ,
                                     "Observed Frequency" : observeds ,
                                     "Normal Theoritical" : theo_normal , 
                                     "Log-Normal Theoritical" : theo_log_normal})
    dataframe["Chi Normal"] = (dataframe["Observed Frequency"]-dataframe["Normal Theoritical"])**2/dataframe["Normal Theoritical"]
    dataframe["Chi Log-Normal"] = (dataframe["Observed Frequency"]-dataframe["Log-Normal Theoritical"])**2/dataframe["Log-Normal Theoritical"]
    return dataframe
def check_chi_square(bins , array , alpha , degree): 
    array = np.array(array)
    limit_x_2 = chi2.ppf(1-alpha , degree)
    dataframe = make_dataframe(array , bins)
    chi_log_nor = dataframe['Chi Log-Normal'].sum()
    chi_nor = dataframe['Chi Normal'].sum()
    answer = ""
    if chi_nor < limit_x_2:
        answer += "Normal : can't say anything\n"
    else:
        answer += "Normal : crossed limit\n"
    if chi_log_nor < limit_x_2:
        answer += "Log Normal : can't say anything\n"
    else:
        answer += "Log Normal : crossed limit\n"
    return_dict = {"Chi Log" : chi_log_nor , "Chi Nor" : chi_nor , "X2 Limit" : limit_x_2}
    return (return_dict , answer)

data = pd.read_csv("../sample_data.csv")
sixty_years_data = list(data['data'].values)
bins = [30,34,38,42,46,50,54]    
print(check_chi_square(bins , sixty_years_data , 0.05,5)[1])

,Interval,Observed Frequency,Normal Theoritical,Log-Normal Theoritical,Chi Normal,Chi Log-Normal
0,< 30,3,8.293884,7.247121,3.379021,2.488994
1,30 - 34,8,5.935442,7.705484,0.718127,0.011257
2,34 - 38,11,7.726177,9.406042,1.387221,0.270114
3,38 - 42,13,8.766112,9.388339,2.044898,1.389393
4,42 - 46,10,8.669247,8.083711,0.204274,0.454267
5,46 - 50,7,7.472874,6.233650,0.029923,0.094213
6,50 - 54,5,5.614671,4.422294,0.067292,0.075469
7,54 >,3,7.521593,7.513359,2.718148,2.711225


Normal : can't say anything
Log Normal : can't say anything

